In [69]:
import pandas as pd

In [70]:
raw_df = pd.read_csv('masterData-2023-08-24.csv')
# raw_df.head()

In [78]:
df = raw_df.drop(columns=['adjclose'])#.set_index(['symbol','label','date'])
df['range'] = df['high'] - df['low']
df['value_type'] = 'nominal'
# df.set_index(["symbol", "label", "date", "open", "high", "low", "close",
            #    "range"], inplace=True)
ratios = {'10': 3.618, '9': 2.618, '8': 1.618, '7': 1.382, '6':1.118, '5':1, '76.2':.764, '4': .618, '50': .5, '3': .382, '2': .236, '1':.118}
curves = ['RC', 'FC']
columns = {'JxD': ['high', -0.5], 'BDP': ['low', 0.382], 'WDP': ['high', -0.382]}
for ratio in ratios.keys():
    columns[f"RC{ratio}"] = ['close', ratios[ratio]]
for ratio in reversed(ratios.keys()):
    columns[f"FC{ratio}"] = ['close', -ratios[ratio]]
# for curve in curves:
#     for ratio in ratios.keys():
#         if curve == 'FC':
#             columns[f"{curve}{ratio}"] = ['close', -ratios[ratio]]
#         else:
#             columns[f"{curve}{ratio}"] = ['close', ratios[ratio]]


In [86]:
for column in columns.keys():
    df[column] = df[columns[column][0]]+df['range']*columns[column][1]

bound_multiplier = 0.073

df_lb = df.copy()
df_lb[list(columns.keys())] = df_lb[list(columns.keys())].sub(df_lb['range']*bound_multiplier, axis='index')
df_lb['value_type'] = 'lower'

df_ub = df.copy()
df_ub[list(columns.keys())] = df_ub[list(columns.keys())].add(df_ub['range']*bound_multiplier, axis='index')
df_ub['value_type'] = 'upper'
# df.head()

df_consolidated = pd.concat([df_lb, df, df_ub])

In [87]:
df_consolidated.set_index(['symbol','label','date', 'open','high','low',
                           'close','range','value_type'], inplace=True)
df_consolidated.sort_index(level=['symbol','label','value_type'])

JxD  \
symbol label   date       open  high  low   close range value_type             
ARX    halfly  2023-06-30 18.05 18.43 14.34 17.67 4.09  lower       16.08643   
                                                        nominal     16.38500   
                                                        upper       16.68357   
       monthly 2023-07-31 17.67 20.19 17.63 19.92 2.56  lower       18.72312   
                                                        nominal     18.91000   
...                                                                      ...   
TRP    weekly  2023-08-18 49.25 49.35 47.93 48.44 1.42  nominal     48.64000   
                                                        upper       48.74366   
       yearly  2022-12-30 59.71 74.44 53.36 53.98 21.08 lower       62.36116   
                                                        nominal     63.90000   
                                                        upper       65.43884   

                                                                         BDP  \
symbol label   date       open  high  low   close range value_type             
ARX    halfly  2023-06-30 18.05 18.43 14.34 17.67 4.09  lower       15.60381   
                                                        nominal     15.90238   
                                                        upper       16.20095   
       monthly 2023-07-31 17.67 20.19 17.63 19.92 2.56  lower       18.42104   
                                                        nominal     18.60792   
...                                                                      ...   
TRP    weekly  2023-08-18 49.25 49.35 47.93 48.44 1.42  nominal     48.47244   
                                                        upper       48.57610   
       yearly  2022-12-30 59.71 74.44 53.36 53.98 21.08 lower       59.87372   
                                                        nominal     61.41256   
                                                        upper       62.95140   

                                                                         WDP  \
symbol label   date       open  high  low   close range value_type             
ARX    halfly  2023-06-30 18.05 18.43 14.34 17.67 4.09  lower       16.56905   
                                                        nominal     16.86762   
                                                        upper       17.16619   
       monthly 2023-07-31 17.67 20.19 17.63 19.92 2.56  lower       19.02520   
                                                        nominal     19.21208   
...                                                                      ...   
TRP    weekly  2023-08-18 49.25 49.35 47.93 48.44 1.42  nominal     48.80756   
                                                        upper       48.91122   
       yearly  2022-12-30 59.71 74.44 53.36 53.98 21.08 lower       64.84860   
                                                        nominal     66.38744   
                                                        upper       67.92628   

                                                                         RC10  \
symbol label   date       open  high  low   close range value_type              
ARX    halfly  2023-06-30 18.05 18.43 14.34 17.67 4.09  lower        32.16905   
                                                        nominal      32.46762   
                                                        upper        32.76619   
       monthly 2023-07-31 17.67 20.19 17.63 19.92 2.56  lower        28.99520   
                                                        nominal      29.18208   
...                                                                       ...   
TRP    weekly  2023-08-18 49.25 49.35 47.93 48.44 1.42  nominal      53.57756   
                                                        upper        53.68122   
       yearly  2022-12-30 59.71 74.44 53.36 53.98 21.08 lower       128.70860   
                                                        nom

In [88]:
df_consolidated.loc['BEP-UN']

,,,,,,,,JxD,BDP,WDP,RC10,RC9,RC8,RC7,RC6,RC5,RC76.2,...,FC3,FC50,FC4,FC76.2,FC5,FC6,FC7,FC8,FC9,FC10
label,date,open,high,low,close,range,value_type,,,,,,,,,,,,,,,,,,,,,
weekly,2023-08-18,35.67,36.19,34.72,35.37,1.47,lower,35.34769,35.17423,35.52115,40.58115,39.11115,37.64115,37.29423,36.90615,36.73269,36.38577,...,34.70115,34.52769,34.35423,34.13961,33.79269,33.61923,33.23115,32.88423,31.41423,29.94423
monthly,2023-07-31,38.77,39.90,38.12,38.42,1.78,lower,38.88006,38.67002,39.09010,44.73010,42.95010,41.17010,40.75002,40.28010,40.07006,39.64998,...,37.61010,37.40006,37.19002,36.93014,36.51006,36.30002,35.83010,35.41002,33.63002,31.85002
quarterly,2023-06-30,42.69,44.13,37.89,39.06,6.24,lower,40.55448,39.81816,41.29080,61.18080,54.94080,48.70080,47.22816,45.58080,44.84448,43.37184,...,36.22080,35.48448,34.74816,33.83712,32.36448,31.62816,29.98080,28.50816,22.26816,16.02816
halfly,2023-06-30,34.96,44.13,34.74,39.06,9.39,lower,38.74953,37.64151,39.85755,72.34755,62.95755,53.56755,51.35151,48.87255,47.76453,45.54849,...,34.78755,33.67953,32.57151,31.20057,28.98453,27.87651,25.39755,23.18151,13.79151,4.40151
yearly,2022-12-30,45.32,53.09,32.58,34.28,20.51,lower,41.33777,38.91759,43.75795,106.98795,86.47795,65.96795,61.12759,55.71295,53.29277,48.45241,...,24.94795,22.52777,20.10759,17.11313,12.27277,9.85259,4.43795,-0.40241,-20.91241,-41.42241
weekly,2023-08-18,35.67,36.19,34.72,35.37,1.47,nominal,35.45500,35.28154,35.62846,40.68846,39.21846,37.74846,37.40154,37.01346,36.84000,36.49308,...,34.80846,34.63500,34.46154,34.24692,33.90000,33.72654,33.33846,32.99154,31.52154,30.05154
monthly,2023-07-31,38.77,39.90,38.12,38.42,1.78,nominal,39.01000,38.79996,39.22004,44.86004,43.08004,41.30004,40.87996,40.41004,40.20000,39.77992,...,37.74004,37.53000,37.31996,37.06008,36.64000,36.42996,35.96004,35.53996,33.75996,31.97996
quarterly,2023-06-30,42.69,44.13,37.89,39.06,6.24,nominal,41.01000,40.27368,41.74632,61.63632,55.39632,49.15632,47.68368,46.03632,45.30000,43.82736,...,36.67632,35.94000,35.20368,34.29264,32.82000,32.08368,30.43632,28.96368,22.72368,16.48368
halfly,2023-06-30,34.96,44.13,34.74,39.06,9.39,nominal,39.43500,38.32698,40.54302,73.03302,63.64302,54.25302,52.03698,49.55802,48.45000,46.23396,...,35.47302,34.36500,33.25698,31.88604,29.67000,28.56198,26.08302,23.86698,14.47698,5.08698


In [14]:
# bounds = ['lb', 'ub']
# bound_multiplier = 0.073
# column_bounds = [f"{column}_{bound}" for bound in bounds
#                                      for column in columns.keys()]
# for column in column_bounds:
#     if 'lb' in column:
#         df[column] = df[column.split("_")[0]] - df['range']*bound_multiplier
#     else:
#         df[column] = df[column.split("_")[0]] + df['range']*bound_multiplier